(data-fundamentals)=
# Data for Discrete Choice

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

import larch as lx

This discussion of data for discrete choice modeling is broken into two parts.  The first part 
is a review of the abstract fundamentals, and is recommended for users new to discrete choice modeling.

If you are are already familar with the various kinds of discrete choice data (case-only and case-alt, 
or wide and tall), you probably want to skip the the second section which focuses on the 
[practical implementation](Practical-Data-Formating-in-Larch) of these formats when used in Larch.

## Fundamental Data Formats

When working with discrete choice models in Larch, we will generally
receive data to input into the system in one of two basic formats: the case-only ("idco")
format or the case-alternative ("idca") format. 

This are sometimes referred to as
IDCase (each record contains all the information for mode choice over
alternatives for a single trip) or IDCase-IDAlt (each record contains all the
information for a single alternative available to each decision maker so there is one
record for each alternative for each choice).

(idco)=
### idco Format

In the **idco** case-only format, each record provides all the relevant information
about an individual choice, including the variables related to the decision maker
or the choice itself, as well as alternative-related variables for all available
alternatives, and a variable indicating which alternative was chosen. This style 
of data has a variety of names in the choice modeling literature, including
"IDCase", "case-only", and "wide".

In [ ]:
data_co = pd.read_csv("example-data/tiny_idco.csv", index_col="caseid")

In [ ]:
data_co

(idca)=
### idca Format

In the [idca](idca) case-alternative format, each record can include information on the variables
related to the decision maker or the choice itself, the attributes of that
particular alternative, and a choice variable that indicates whether the
alternative was or was not chosen. This style of data has a variety of names in the 
choice modeling literature, including "IDCase-IDAlt", "case-alternative", and "tall".

In [ ]:
data_ca = pd.read_csv("example-data/tiny_idca.csv")

In [ ]:
data_ca

(idce)=
#### sparse vs dense

The `idca` format actually has two technical variations, a sparse version and a 
dense version. The table shown above is a sparse version, where any alterative that 
is not available is simply missing from the data table.  Thus, in caseid 2 above, 
there are only 2 rows, not 3.  By dropping these rows, this data storage is potentially
more efficient than the dense version.  But, in cases where the number of missing alternatives
is managably small (less than half of all the data, certainly) it can be much more computationally
efficient to simply store and work with the dense array. 

In *Larch*, these two distinct sub-types of idca data are labeled so 
that the dense version labeled as `idca` and the sparse version 
labeled as `idce`. 

### Data Conversion

Converting between `idca` format data and `idco` format in Python can be super easy if the alternative
id's are stored appropriately in a two-level MultiIndex. In that case, we can simply `stack` or `unstack` the DataFrame, and change formats.  This is typically more readily available when switching from `idca` to `idco`
formats, as the alterative id's typically appear in a column of the DataFrame that can be used for indexing.

In [ ]:
data_ca.set_index(["caseid", "altid"]).unstack()

Getting our original `idco` data into `idca` format is not so clean, as there's no analagous
`set_columns` method in pandas, and even if there were, the alternative codes are not typically
neatly arranged in a row of data. We can force it to work, but it's not pretty.

In [ ]:
forced_ca = data_co.T.set_index(
    pd.MultiIndex.from_tuples(
        [
            ["Car", "Income"],
            ["Car", "Time"],
            ["Car", "Cost"],
            ["Bus", "Time"],
            ["Bus", "Cost"],
            ["Walk", "Time"],
            ["Car", "Chosen"],
        ],
        names=("alt", "var"),
    )
).T.stack(0)
forced_ca[["Chosen", "Income"]] = (
    forced_ca[["Chosen", "Income"]]
    .groupby("caseid")
    .transform(lambda x: x.fillna(x.value_counts().index[0]))
)
forced_ca["Chosen"] = (
    forced_ca["Chosen"] == forced_ca.index.get_level_values("alt")
).astype(float)
forced_ca

## Practical Data Formating in Larch

The data formats described above are relevant when storing data in 
a tabular (two-dimensional) format.  This is quite common and generally
expected, especially for data exchange between most software tools,
but Larch doesn't require you to choose one or the other.

Instead, Larch uses a [`Dataset`](larch.dataset.Dataset) structure based
on `xarray`, to store and use a collection of relevant variables, and 
each variable can be stored in either [idco](idco) or [idca](idca) format, as 
appropriate.

In [ ]:
dataset = lx.dataset.merge(
    [
        data_co[["Income", "Chosen"]].to_xarray(),
        data_ca.set_index(["caseid", "altid"])[["Time", "Cost"]].to_xarray(),
    ],
    caseid="caseid",
    alts="altid",
)
dataset

As we saw above, it's quite easy to move from [idca](idca) to [idco](idco) format,
and Larch can apply those transformations automatically when loading [idca](idca)
data, using the [`Dataset.construct.from_idca`](`Dataset.construct.from_idca`) method.  
In the example below, note that the `Income` variable has automatically
been collapsed to [idco](idco), while the other variables remain as [idca](idca).

In [ ]:
lx.Dataset.construct.from_idca(
    data_ca.set_index(["caseid", "altid"]),
)

In [ ]:
# TEST
from pytest import approx

t = lx.Dataset.construct.from_idca(
    data_ca.set_index(["caseid", "altid"]),
)

assert all(t.altid == [1, 2, 3])
assert all(t.alt_names == ["Bus", "Car", "Walk"])
assert t.Cost.where(t._avail_, 0).data == approx(
    np.array(
        [
            [100, 150, 0],
            [100, 125, 0],
            [75, 125, 0],
            [150, 225, 0],
        ]
    )
)
assert t.Time.where(t._avail_, 0).data == approx(
    np.array(
        [
            [40, 30, 20],
            [35, 25, 0],
            [50, 40, 30],
            [20, 15, 10],
        ]
    )
)

Loading data in sparse format is as easy as swapping out 
[`from_idca`](larch.Dataset.from_idca) for 
[`from_idce`](larch.Dataset.from_idce).  The resulting
dataset will have a similar collection of variables, but 
each idca variable is stored in a one-dimensional array,
using a variety of the compressed sparse row data format.

In [ ]:
lx.Dataset.construct.from_idce(
    data_ca.set_index(["caseid", "altid"]),
)

In [ ]:
# TEST
z = lx.Dataset.construct.from_idce(
    data_ca.set_index(["caseid", "altid"]),
)
assert z.Income.dims == ("caseid",)
assert z.Time.dims == ("_casealt_",)
assert z["_caseptr_"].shape == (5,)
assert all(z["_caseptr_"] == [0, 3, 5, 8, 11])

## Data Encoding

For the most part, data used in the utility functions of discrete choice models enters into the utility function as part of a linear-in-parameters function.  That is, we have some "data" that expresses an attribute of some part of the transportation system as a number, we multiply that by some numerical parameter that will be estimated, and we sum up the total over all the data-times-parameter operations.  This kind of structure is known as "linear algebra" and it's something computers can do super fast, as long as all the data and all the parameters are queued up in memory in the right formats.  So, typically it is optimal to pre-compute the "data" part of the process into one large contiguous array of floating point values, regardless if the values otherwise seem to be binary or integers. Most tools, such as Larch, will do much of this work for you, so you don't need to worry about it too much.   

There are two notable exceptions to this guideline: 

- *choices*: the data that represents the observed choices, which are inherently categorical
- *availability*: data that represents the availability of each choice, which is inherently boolean

### Categorical Encoding

When we are looking at discrete choices, it is natural to employ a categorical data type for at least the "choice" data itself, if not for other columns as well.  Pandas can convert columns to categorical data simply by assigning the type "category".

In [ ]:
choices = data_co["Chosen"].astype("category")
choices

Once we have categorical data, if we like we can work with the underlying code values instead of the original raw data.

In [ ]:
choices.cat.codes

The `cat.categories` attribute contains the array of values matching each of the code.

In [ ]:
choices.cat.categories

When using `astype("category")` there's no control over the ordering of the categories.  If we want
to control the apparent order (e.g. we already have codes defined elsewhere such that Car is 1, Bus is 2, and walk is 3) then we can explicitly set the category value positions using `pd.CategoricalDtype` instead of `"category"`.
Note that the `cat.codes` numbers used internally by categoricals start with zero as standard in Python,
so if you want codes to start with 1 you need to include a dummy placeholder for zero.

In [ ]:
choices1 = data_co["Chosen"].astype(pd.CategoricalDtype(["_", "Car", "Bus", "Walk"]))
choices1

In [ ]:
choices1.cat.codes

To be clear, by asserting the *placement* ordering of alternative like this, we are not simultaneously asserting that the alternatives are ordinal.  Put another way, we are forcing Car to be coded as 1 and Bus to be coded as 2, but we are not saying that Car is less than Bus.  Pandas categoricals can allow this, by adding `ordered=True` to the CategoricalDtype. 

In [ ]:
pd.CategoricalDtype(["NoCars", "1Car", "2Cars", "3+Cars"], ordered=True)

### One Hot Encoding

One-hot encoding, also known as dummy variables, is the creation of a seperate binary-valued column for every categorical value.  We can convert a categorical data column into a set of one-hot encoded columns using the `get_dummies` function.

In [ ]:
pd.get_dummies(choices)

It's not required to have first converted the data to a categorical data type.

In [ ]:
pd.get_dummies(data_co["Chosen"])

### Encoding with `xarray`

The `xarray` library doesn't use formal "categorical" datatypes, but we can still use
the `get_dummies` function to explode choice and availability data as needed.

In [ ]:
dataset["Chosen_ca"] = lx.DataArray(
    pd.get_dummies(data_co["Chosen"]).rename_axis(columns="altid")
)

In [ ]:
dataset

## Working with Skims

Sometimes, data used by Larch for estimating transportation choices is available as "skims".
This data is usually stored in one of two formats: open matrix files, or indexed csv tables.
Both formats are readily usable by models in Larch.  For open matrix files, there is the 
`OMX` class, which is derived from a PyTables File object and allows you to open and refer to
open matrix files on disk.  When you first access an open matrix file, only the meta-data is
actually read, allowing you to see what is in the file before actually loading the whole thing
into memory.

In [ ]:
from larch.omx import OMX

skims = OMX(lx.example_file("exampville_skims.omx"), mode="r")
skims

It is easy to convert an OMX object into a dataset:

In [ ]:
skims_dataset = skims.to_dataset()
skims_dataset

The other common (although less efficient) format for skim data is an indexed table.
This could be a csv file, with "X" and "Y" coordinates given in a pair of columns, and
one or more variables in other columns.  Data in this format might look something like 
this:

In [ ]:
skim_df = pd.read_csv("example-data/exampville_skims.csv")
skim_df

As shown above, loading this format of data into a pandas DataFrame is as simple 
as reading the CSV file. in the usual way.  Converting it to an xarray Dataset is
also easy:

In [ ]:
skim_df.set_index(["I", "J"]).to_xarray()

If you're going to be reading and writing the same datasets over and over, it 
can be advantageous to store them in a format that's more efficient for reading and 
writing, such as the ZARR format.  You can write a dataset to zarr using `Dataset.to_zarr`:

In [ ]:
skims_dataset.to_zarr("example-data/example-skims.zarr", mode="w")

Opening a dataset that has been saved in ZARR format is possible using the `open_zarr` command.

In [ ]:
z = xr.open_zarr("example-data/example-skims.zarr")
z

The `open_zarr` function returns promptly because, like the OMX class, 
by default it does not actually read the underlying data into memory, just the 
meta-data that identifies variable names, shapes, and types.  Actually loading the
data itself can be deferred until it is actually needed, or it can be read eagerly
using the `load` method.  You see below the *dask.array* data elements have been replaced
with actual values.

In [ ]:
z.load()